In [1]:
from llama_index.llms.ollama import Ollama
from utils.rag import *

llm = Ollama(model="gemma:2b", request_timeout=60.0)
collection=Ingestion(path='data/SPOTIFY_REVIEWS.csv',embedding_model="BAAI/bge-small-en-v1.5",rowlimit=2000)
collection.run()
bot=RAGBot(collection=collection,MODEL_NAME="gemma:2b",REQUEST_TIMEOUT=60)

No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.2'


Data ingestion


c:\Users\nurro\.conda\envs\tf_gpu\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
questions=[
    "What are the specific features or aspects that users appreciate the most in our application?",
    "In comparison to our application, which music streaming platform are users most likely to compare ours with?",
    "What are the primary reasons users express dissatisfaction with Spotify?",
    "Can you identify emerging trends or patterns in recent user reviews that may impact our product strategy?",
    ]
queries=[]
answers=[]
for que in questions:
    que,answ=bot.qna(question=que,RAG_KEYWORD=60,get_query=True)
    queries.append(que)
    answers.append(answ)

inference process
inference process
inference process
inference process


In [40]:
for i in range(len(questions)):
    with open(f'data/eval/answer_rel_{i}.txt', 'w') as file:
        a0=f'''
Your task is to evaluate if the response is relevant to the query.
The evaluation should be performed in a step-by-step manner by answering the following questions:
1. Does the provided response match the subject matter of the user's query?
2. Does the provided response attempt to address the focus or perspective
on the subject matter taken on by the user's query?
Each question above is worth 1 point. Provide detailed feedback on response according to the criteria questions above
After your feedback provide a final result by strictly following this format: '[RESULT] followed by the integer number representing the total score assigned to the response'
Query: {queries[i]}
Response: {answers[i]}
Feedback:
'''
        # Write the text to the file
        file.write(a0)

    with open(f'data/eval/context_rel_{i}.txt', 'w') as file:
        a1=f'''
Your task is to evaluate if the retrieved context from the document sources are relevant to the query
The evaluation should be performed in a step-by-step manner by answering the following questions:
1. Does the retrieved context match the subject matter of the user's query?
2. Can the retrieved context be used exclusively to provide a full answer to the user's query?
Each question above is worth 2 points, where partial marks are allowed and encouraged. Provide detailed feedback on the response
according to the criteria questions previously mentioned.
After your feedback provide a final result by strictly following this format: 
[RESULT] followed by the float number representing the total score assigned to the response
Query: {questions[i]}
Context: {queries[i]}
Feedback:
'''
        # Write the text to the file
        file.write(a0)